<h1>Vorhersage, ob es bei einem Unfall Schwerverletzte/Tote gibt</h1>

## 1. Vorbereitung des Datensatzes

### 1.1 Generelles Aufbereiten der Datenbasis

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
crash_data = gpd.read_file('../01_Source/U_VM_A_L_BRW_TL.geojson') 
geo_information = pd.read_csv('../01_Source/geo_information.csv', sep=';')
crash_data = pd.merge(crash_data, geo_information, on ='OBJECTID',  how='inner')

In [2]:
crash_data.head()

,OBJECTID,ULAND,UREGBEZ,UKREIS,UGEMEINDE,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,UKATEGORIE,UART,UTYP1,ULICHTVERH,IstRad,IstPKW,IstFuss,IstKrad,IstGkfz,IstSonstige,STRZUSTAND,LINREFX,LINREFY,METER,STR_NAME,DTV,PKW,LKW,LIEFERWAGEN,LINIENBUSSE,REISEBUSSE,KRAD,HERKUNFT,NAME,KNAME,PLR_NAME,EW2010,VBUS,VBUS_PERS,GESLAERM,GESLAERM_PERS,GEMEINDE,BEZIRK,ANWERT,BRW,NUTZUNG,GFZ,VERFAHRENSART,STICHTAG,BEITRAGSZUSTAND,LUMNUM,ELEM_NR,VRICHT_TXT,WERT_VES,ZEIT_T,TAG_T,DURCH_T,DANN_T,DAT_T,geometry,LAT,LON,formatted_address,address_components,postal_code,sublocality,street_number,route
0,198728,11,0,10,10,2018,3,11,5,3,5,3,0,0,1,0,0,0,0,0,814987.3095,5.829184e+06,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,Mahlsdorf Nord,8818.0,mittel,niedrig - sehr niedrig,mittel,niedrig - sehr niedrig,Berlin,Marzahn-Hellersdorf,None,360,W - Wohngebiet,0.4,None,2019-01-01T00:00:00,Beitragsfrei nach BauGB,[[url]],62540035_62540036.02,beide Richtungen,30.0,None,None,angeordnete Verkehrseinschränkung,None,None,"MULTIPOLYGON (((814997.310 5829184.012, 814996...",52.521509,13.643731,"Schongauerstraße 20, 12623 Berlin, Germany","[{""long_name"": ""20"", ""short_name"": ""20"", ""type...",12623.0,Bezirk Marzahn-Hellersdorf,20,Schongauerstraße
1,196865,11,0,10,10,2018,1,18,2,3,5,3,2,0,1,0,0,0,0,0,814875.2039,5.830058e+06,220.0,Lemkestr.,6108.0,4995.0,220.0,624.0,160.0,16.0,93.0,"Modellrechnung 2014, VMZ",None,None,Mahlsdorf Nord,8818.0,mittel,niedrig - sehr niedrig,mittel,niedrig - sehr niedrig,Berlin,Marzahn-Hellersdorf,None,360,W - Wohngebiet,0.4,None,2019-01-01T00:00:00,Beitragsfrei nach BauGB,[[url]],62550001_62550021.01,beide Richtungen,30.0,07:00 - 17:00,Montag bis Freitag,Kinder,Zeitangabe,None,"MULTIPOLYGON (((814885.204 5830057.780, 814884...",52.529404,13.642912,"Lemkestraße 112B, 12623 Berlin, Germany","[{""long_name"": ""112B"", ""short_name"": ""112B"", ""...",12623.0,Bezirk Marzahn-Hellersdorf,112B,Lemkestraße
2,202456,11,0,10,10,2018,6,7,4,2,5,3,0,1,1,0,0,0,0,0,814271.9549,5.828041e+06,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,Alt-Mahlsdorf,3852.0,niedrig - sehr niedrig,niedrig - sehr niedrig,niedrig - sehr niedrig,mittel,Berlin,Marzahn-Hellersdorf,None,360,W - Wohngebiet,0.4,None,2019-01-01T00:00:00,Beitragsfrei nach BauGB,[[url]],62530003_62540023.02,beide Richtungen,30.0,None,None,angeordnete Verkehrseinschränkung,None,None,"MULTIPOLYGON (((814281.955 5828040.742, 814281...",52.511678,13.632143,"Lortzingstraße 22, 12623 Berlin, Germany","[{""long_name"": ""22"", ""short_name"": ""22"", ""type...",12623.0,Bezirk Marzahn-Hellersdorf,22,Lortzingstraße
3,201726,11,0,10,10,2018,6,13,4,3,5,3,0,0,1,0,0,1,0,0,814205.0080,5.830068e+06,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,Mahlsdorf Nord,8818.0,mittel,niedrig - sehr niedrig,mittel,niedrig - sehr niedrig,Berlin,Marzahn-Hellersdorf,None,360,W - Wohngebiet,0.4,None,2019-01-01T00:00:00,Beitragsfrei nach BauGB,[[url]],62550008_62550009.03,beide Richtungen,30.0,None,None,angeordnete Verkehrseinschränkung,None,None,"MULTIPOLYGON (((814215.008 5830068.156, 814214...",52.529883,13.633075,"Am Schlehdorn 24, 12623 Berlin, Germany","[{""long_name"": ""24"", ""short_name"": ""24"", ""type...",12623.0,Bezirk Marzahn-Hellersdorf,24,Am Schlehdorn
4,200478,11,0,10,10,2018,5,20,7,3,5,3,1,0,1,0,0,0,0,0,814755.4276,5.829893e+06,254.0,Lemkestr.,3852.0,3127.0,107.0,393.0,150.0,16.0,59.0,"Modellrechnung 2014, VMZ",None,None,Mahlsdorf Nord,8818.0,mittel,niedrig - sehr niedrig,mittel,niedrig - sehr niedrig,Berlin,Marzahn-Hellersdorf,None,360,W - Wohngebiet,0.4,None,2019-01-01T00:00:00,Beitragsfrei nach BauGB,[[url]],62550028_62550029.02,beide Richtungen,30.0,None,None,angeordnete Verkehrseinschränkung,None,None,"MULTIPOLYGON (((814765.428 5829893.128, 814764...",52.527997,13.640996,"Lemkestr./Greifswalder Str. (Berlin), 12623 Be...","[{""long_name"": ""Lemkestr./Greifswalder Str. (B...",12623.0,Bezirk Marzahn-Hellersdorf,NaN,Lemkestr./Greifswalder Str. (Berlin)


In [3]:
crash_data.describe()

,OBJECTID,ULAND,UREGBEZ,UKREIS,UGEMEINDE,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,UKATEGORIE,UART,UTYP1,ULICHTVERH,IstRad,IstPKW,IstFuss,IstKrad,IstGkfz,IstSonstige,STRZUSTAND,LINREFX,LINREFY,METER,DTV,PKW,LKW,LIEFERWAGEN,LINIENBUSSE,REISEBUSSE,KRAD,EW2010,GFZ,WERT_VES,LAT,LON,postal_code
count,13649.000000,13649.0,13649.0,13649.000000,13649.000000,13649.0,13649.000000,13649.000000,13649.000000,13649.000000,13649.000000,13649.000000,13649.000000,13649.000000,13649.000000,13649.000000,13649.000000,13649.000000,13649.000000,13649.000000,13649.000000,1.364900e+04,11689.000000,11689.000000,11689.000000,11689.000000,11689.000000,11689.000000,11689.000000,11689.000000,13313.000000,11114.000000,8388.000000,13649.000000,13649.000000,13609.000000
mean,190757.504506,11.0,0.0,5.424793,5.424793,2018.0,6.728405,13.473441,4.057513,2.834640,3.626713,3.887171,0.446699,0.380321,0.817056,0.151147,0.157814,0.032457,0.133050,0.195472,797784.714863,5.826565e+06,212.232355,19575.767816,16216.331679,580.265549,1996.719309,251.738814,67.300967,463.459406,9840.139037,1.931591,41.808417,52.507661,13.388595,12002.414726
std,30324.889510,0.0,0.0,3.495025,3.495025,0.0,3.175898,4.773724,1.779208,0.379519,2.147327,1.895501,0.796455,0.485483,0.386635,0.358205,0.364580,0.177216,0.339641,0.408232,6614.330070,5.157698e+03,241.785709,13428.201798,11202.698673,607.305704,1369.688024,275.290243,49.560774,317.187615,5899.990231,1.159272,787.939231,0.046722,0.096822,1330.929794
min,112695.000000,11.0,0.0,1.000000,1.000000,2018.0,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,778444.698700,5.809746e+06,12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,321.000000,0.100000,5.000000,52.346532,13.094676,10115.000000
25%,198395.000000,11.0,0.0,2.000000,2.000000,2018.0,4.000000,10.000000,3.000000,3.000000,2.000000,2.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,793462.791500,5.823267e+06,110.000000,9961.000000,8200.000000,224.000000,1016.000000,20.000000,32.000000,240.000000,5735.000000,1.000000,30.000000,52.478603,13.326103,10627.000000
50%,201810.000000,11.0,0.0,5.000000,5.000000,2018.0,7.000000,14.000000,4.000000,3.000000,5.000000,3.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,797762.199400,5.826844e+06,167.000000,16470.000000,13560.000000,414.000000,1680.000000,200.000000,64.000000,388.000000,9236.000000,2.000000,30.000000,52.510399,13.387512,12203.000000
75%,205222.000000,11.0,0.0,8.000000,8.000000,2018.0,9.000000,17.000000,6.000000,3.000000,5.000000,6.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,801550.244200,5.829905e+06,254.000000,25650.000000,21422.000000,722.000000,2616.000000,370.000000,98.000000,616.000000,12958.000000,2.500000,30.000000,52.538200,13.444049,13347.000000
max,208853.000000,11.0,0.0,12.000000,12.000000,2018.0,12.000000,23.000000,7.000000,3.000000,9.000000,7.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,822060.509200,5.842633e+06,6418.000000,89544.000000,74421.000000,4780.000000,9133.000000,2560.000000,406.000000,2240.000000,31699.000000,4.500000,72189.000000,52.655866,13.738100,16816.000000


In [4]:
crash_data = crash_data.drop(["ULAND","UREGBEZ","UJAHR","LINREFX","LINREFY","METER","STR_NAME","EW2010","GFZ","HERKUNFT","NAME","KNAME","PLR_NAME",
                              "VBUS","VBUS_PERS","GESLAERM_PERS","GEMEINDE","BEZIRK","ANWERT","VERFAHRENSART","STICHTAG","BEITRAGSZUSTAND","LUMNUM","ELEM_NR","VRICHT_TXT","ZEIT_T","TAG_T"
                             ,"DURCH_T","DANN_T","DAT_T","LAT","LON","formatted_address","address_components","street_number","route","postal_code","sublocality","BRW"], axis=1)

In [5]:
# Aus UKategorie das Target extrahieren
# Wenn UKategorie = 1 oder 2 dann target 1 sonst 0
# Unfallart


crash_data["UKATEGORIE"] = crash_data["UKATEGORIE"]/2
crash_data['TARGET'] = 0 #0 leicht verletzt 1= Schwer Verletzt/getötet
crash_data.loc[crash_data["UKATEGORIE"] <= 1, ['TARGET']] = 1


def id_switcher(left_df, right_df):
    """

    :param left_df: main df
    :param right_df: merger df, col 1 = merger column; col 2 corresponding value
    :return: dataframe
    """
    on_column = right_df.columns[0]
    on_column_value = on_column + "_Value"
    left_df[on_column_value] = left_df.merge(right_df, on = on_column)['Value']
    left_df = left_df.drop([on_column],axis=1)
    return left_df


uart_df = pd.DataFrame({'UART' : [1,2,3,4,5,6,7,8,9,10], 'Value':["Zusammenstoß mit anfahrendem / anhaltendem / ruhendem Fahrzeug",
                "Zusammenstoß mit vorausfahrendem / wartendem Fahrzeug",
                "Zusammenstoß mit seitlich in gleicher Richtung fahrendem Fahrzeug",
                "Zusammenstoß mit entgegenkommendem Fahrzeug",
                "Zusammenstoß mit einbiegendem / kreuzendem Fahrzeug",
                "Zusammenstoß zwischen Fahrzeug und Fußgänger",
                "Aufprall auf Fahrbahnhindernis",
                "Abkommen von Fahrbahn nach rechts",
                "Abkommen von Fahrbahn nach links",
                "Unfall anderer Art"]})

crash_data = id_switcher(crash_data, uart_df)



# Wochentag

wochentag_df = pd.DataFrame({'UWOCHENTAG':[1,2,3,4,5,6,7],
                                 'Value':['Sonntag','Montag','Dienstag','Mittwoch','Donnerstag','Freitag','Samstag']})
crash_data = id_switcher(crash_data, wochentag_df)
    

unfalltyp_df = pd.DataFrame({'UTYP1':[1,2,3,4,5,6,7],
                                 'Value':['Fahrunfall','Abbiegeunfall','Einbiegen / Kreuzen-Unfall','Überschreitenunfall','Unfall durch ruhenden Verkehr','Unfall im Längsverkehr','sonstiger Unfall']})
crash_data = id_switcher(crash_data, unfalltyp_df)

licht_df = pd.DataFrame({'ULICHTVERH':[0,1,2],
                                 'Value':['Tageslicht','Dämmerung','Dunkelheit']})
crash_data = id_switcher(crash_data, licht_df)

str_zustand_df = pd.DataFrame({'STRZUSTAND':[0,1,2],
                                 'Value':['trocken','nass/feucht/schlüpfrig','winterglatt']})
crash_data = id_switcher(crash_data, str_zustand_df)

crash_data['DTV'] = crash_data['DTV'].fillna(crash_data['DTV'].mean())
crash_data["ANTEIL_PKW"] = crash_data["PKW"]*100/crash_data["DTV"]
crash_data["ANTEIL_PKW"] = crash_data["ANTEIL_PKW"].fillna(crash_data["ANTEIL_PKW"].mean())
crash_data["ANTEIL_LKW"] = crash_data["LKW"]*100/crash_data["DTV"]
crash_data["ANTEIL_LKW"] = crash_data["ANTEIL_LKW"].fillna(crash_data["ANTEIL_LKW"].mean())
crash_data["ANTEIL_LIEFERWAGEN"] = crash_data["LIEFERWAGEN"]*100/crash_data["DTV"]
crash_data["ANTEIL_LIEFERWAGEN"] = crash_data["ANTEIL_LIEFERWAGEN"].fillna(crash_data["ANTEIL_LKW"].mean())
crash_data["ANTEIL_LINIENBUSSE"] = crash_data["LINIENBUSSE"]*100/crash_data["DTV"]
crash_data["ANTEIL_LINIENBUSSE"] = crash_data["ANTEIL_LINIENBUSSE"].fillna(crash_data["ANTEIL_LKW"].mean())
crash_data.loc[crash_data["ANTEIL_LINIENBUSSE"] == np.inf, ['DTV']] = crash_data.loc[crash_data["ANTEIL_LINIENBUSSE"] == np.inf, ['LINIENBUSSE']]
crash_data.loc[crash_data["ANTEIL_LINIENBUSSE"] == np.inf, ['ANTEIL_LINIENBUSSE']] = 100
crash_data["ANTEIL_REISEBUSSE"] = crash_data["REISEBUSSE"]*100/crash_data["DTV"]
crash_data["ANTEIL_REISEBUSSE"] = crash_data["ANTEIL_REISEBUSSE"].fillna(crash_data["ANTEIL_REISEBUSSE"].mean())
crash_data["ANTEIL_KRAD"] = crash_data["KRAD"]*100/crash_data["DTV"]
crash_data["ANTEIL_KRAD"] = crash_data["ANTEIL_KRAD"].fillna(crash_data["ANTEIL_KRAD"].mean())
crash_data = crash_data.drop(["PKW","LKW","LIEFERWAGEN","LINIENBUSSE","REISEBUSSE","KRAD"], axis=1)

In [6]:
crash_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 13649 entries, 0 to 13648
Data columns (total 29 columns):
OBJECTID              13649 non-null int64
UKREIS                13649 non-null int64
UGEMEINDE             13649 non-null int64
UMONAT                13649 non-null int64
USTUNDE               13649 non-null int64
UKATEGORIE            13649 non-null float64
IstRad                13649 non-null int64
IstPKW                13649 non-null int64
IstFuss               13649 non-null int64
IstKrad               13649 non-null int64
IstGkfz               13649 non-null int64
IstSonstige           13649 non-null int64
DTV                   13647 non-null float64
GESLAERM              13258 non-null object
NUTZUNG               12581 non-null object
WERT_VES              8388 non-null float64
geometry              13649 non-null geometry
TARGET                13649 non-null int64
UART_Value            12058 non-null object
UWOCHENTAG_Value      13649 non-null object
UTYP1_Value

In [7]:
crash_data = crash_data.drop(["UKATEGORIE","OBJECTID"], axis=1)
crash_data.head()

,UKREIS,UGEMEINDE,UMONAT,USTUNDE,IstRad,IstPKW,IstFuss,IstKrad,IstGkfz,IstSonstige,DTV,GESLAERM,NUTZUNG,WERT_VES,geometry,TARGET,UART_Value,UWOCHENTAG_Value,UTYP1_Value,ULICHTVERH_Value,STRZUSTAND_Value,ANTEIL_PKW,ANTEIL_LKW,ANTEIL_LIEFERWAGEN,ANTEIL_LINIENBUSSE,ANTEIL_REISEBUSSE,ANTEIL_KRAD
0,10,10,3,11,0,1,0,0,0,0,19575.767816,mittel,W - Wohngebiet,30.0,"MULTIPOLYGON (((814997.310 5829184.012, 814996...",0,Zusammenstoß mit einbiegendem / kreuzendem Fah...,Donnerstag,Einbiegen / Kreuzen-Unfall,Tageslicht,trocken,82.408776,2.762958,2.762958,2.762958,0.400124,2.494525
1,10,10,1,18,0,1,0,0,0,0,6108.000000,mittel,W - Wohngebiet,30.0,"MULTIPOLYGON (((814885.204 5830057.780, 814884...",0,Zusammenstoß mit einbiegendem / kreuzendem Fah...,Donnerstag,Einbiegen / Kreuzen-Unfall,Tageslicht,trocken,81.777996,3.601834,10.216110,2.619515,0.261952,1.522593
2,10,10,6,7,1,1,0,0,0,0,19575.767816,niedrig - sehr niedrig,W - Wohngebiet,30.0,"MULTIPOLYGON (((814281.955 5828040.742, 814281...",1,Zusammenstoß mit einbiegendem / kreuzendem Fah...,Donnerstag,Einbiegen / Kreuzen-Unfall,Tageslicht,trocken,82.408776,2.762958,2.762958,2.762958,0.400124,2.494525
3,10,10,6,13,0,1,0,0,1,0,19575.767816,mittel,W - Wohngebiet,30.0,"MULTIPOLYGON (((814215.008 5830068.156, 814214...",0,Zusammenstoß mit einbiegendem / kreuzendem Fah...,Donnerstag,Einbiegen / Kreuzen-Unfall,Tageslicht,trocken,82.408776,2.762958,2.762958,2.762958,0.400124,2.494525
4,10,10,5,20,0,1,0,0,0,0,3852.000000,mittel,W - Wohngebiet,30.0,"MULTIPOLYGON (((814765.428 5829893.128, 814764...",0,Zusammenstoß mit einbiegendem / kreuzendem Fah...,Donnerstag,Einbiegen / Kreuzen-Unfall,Tageslicht,trocken,81.178609,2.777778,10.202492,3.894081,0.415369,1.531672


In [8]:


#DTV,GESLAERM,BRW, NUTZUNG,WERT_VES,
crash_data.loc[crash_data["WERT_VES"] > 100, ['WERT_VES']] = crash_data["WERT_VES"].mean()
crash_data['WERT_VES'] = crash_data['WERT_VES'].fillna(crash_data['WERT_VES'].mean())
crash_data['UART_Value'] = crash_data['UART_Value'].fillna("Unbekannt")
crash_data['GESLAERM'] = crash_data['GESLAERM'].fillna("mittel")



In [9]:
crash_data_copy = crash_data.copy()
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["UART_Value"], prefix='UART_Value_')
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["UWOCHENTAG_Value"], prefix='UWOCHENTAG_Value_')
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["UTYP1_Value"], prefix='UTYP1_Value_')
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["ULICHTVERH_Value"], prefix='ULICHTVERH_Value_')
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["STRZUSTAND_Value"], prefix='STRZUSTAND_Value_')
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["NUTZUNG"], prefix='NUTZUNG_')
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["GESLAERM"], prefix='GESLAERM_')

corr_matrix = crash_data_copy.corr()
corr_matrix["TARGET"]

UKREIS                                                                           0.052677
UGEMEINDE                                                                        0.052677
UMONAT                                                                          -0.009643
USTUNDE                                                                          0.002866
IstRad                                                                          -0.040838
IstPKW                                                                          -0.062998
IstFuss                                                                          0.112072
IstKrad                                                                          0.073179
IstGkfz                                                                          0.032598
IstSonstige                                                                     -0.011020
DTV                                                                             -0.012357
WERT_VES  

In [10]:
crash_data_copy['DTV'] = crash_data_copy['DTV'].fillna(crash_data_copy['DTV'].mean())
crash_data_copy = crash_data_copy.drop(["geometry"], axis=1)
crash_data_copy.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 13649 entries, 0 to 13648
Data columns (total 62 columns):
UKREIS                                                                           13649 non-null int64
UGEMEINDE                                                                        13649 non-null int64
UMONAT                                                                           13649 non-null int64
USTUNDE                                                                          13649 non-null int64
IstRad                                                                           13649 non-null int64
IstPKW                                                                           13649 non-null int64
IstFuss                                                                          13649 non-null int64
IstKrad                                                                          13649 non-null int64
IstGkfz                                                            

In [11]:

# Import train_test_split function
from sklearn.model_selection import train_test_split

X = crash_data_copy.drop(["TARGET"] , axis=1)
Y = crash_data_copy["TARGET"]
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=42)

In [31]:

#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=5000, max_depth=1)

param_grid = {
                 'n_estimators': [5, 15, 30, 100, 500, 1000],
                 'max_depth': [1, 2, 5, 7, 9, 13]
             }

from sklearn.model_selection import GridSearchCV

#grid_clf = GridSearchCV(clf, param_grid, cv=10)
#grid_clf.fit(X_train, y_train)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

In [32]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.836996336996337


In [33]:
#grid_clf.best_params_
#{'max_depth': 1, 'n_estimators': 5}

In [34]:
feature_imp = pd.Series(clf.feature_importances_, index= X_train.columns).sort_values(ascending=False)

In [35]:
feature_imp

IstFuss                                                                          0.1104
IstPKW                                                                           0.0820
IstKrad                                                                          0.0808
UKREIS                                                                           0.0798
UGEMEINDE                                                                        0.0790
ANTEIL_KRAD                                                                      0.0760
ANTEIL_REISEBUSSE                                                                0.0492
ANTEIL_LIEFERWAGEN                                                               0.0474
USTUNDE                                                                          0.0456
IstRad                                                                           0.0424
ANTEIL_LKW                                                                       0.0390
IstGkfz                         

In [18]:
crash_data_copy["TARGET"].value_counts()

0    11433
1     2216
Name: TARGET, dtype: int64

In [20]:
(11433-2216)/11433

0.806175107145981